In [2]:
import sys
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras

In [3]:
tf.__version__

'2.0.0'

In [5]:
emotion =  ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

model = tf.keras.models.load_model("model.h5")
font = cv2.FONT_HERSHEY_SIMPLEX
cam = cv2.VideoCapture(0)
face_cas = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    # capture frame-by-frame
    ret, frame = cam.read()

    if ret == True:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cas.detectMultiScale(gray, 1.3, 5)

    
        for (x, y, w, h) in faces:
            face_component = gray[y:y+h, x:x+w]
            # reshape into 48x48 pixel image input for model
            fc = cv2.resize(face_component, (48, 48))
            inpt = np.reshape(fc, (1, 48, 48, 1)).astype(np.float32)
            inpt = inpt/255.
            prediction = model.predict_proba(inpt)
            em = emotion[np.argmax(prediction)]
            score = np.max(prediction)
            if score > 0.50:
                cv2.putText(frame, em +"  "+ str(round(score*100, 1)) +'%', (x, y), font, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.imshow("image", frame)
        
        # exit the script if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            final_em = em
            print("Last emotion detected: " + em)
            break
            
    else:
        print ('Error')

cam.release()

cv2.destroyAllWindows()

Last emotion detected: Neutral
